# Descarga de librerias:

In [3]:
# pip install pandas transformers langchain langchain-community torch faiss-cpu

In [4]:
import random
import PyPDF2
import datetime
from pathlib import Path
# import pdfplumber
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

2025-07-13 19:03:10.462691: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752433390.800420      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752433390.894754      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Lectura de archivos:

In [ ]:

# ─────── CONFIGURACIÓN GENERAL ───────
PDF_PATH = Path(r"C:\Users\user\Documents\chatbot\riesgo-credito-2.pdf")
EMBEDDING_MODEL = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
LLM_MODEL = "google/flan-t5-large"

# ─────── CHISTES FINANCIEROS (opcional) ───────
financial_jokes = [
    "¿Por qué el banco no le prestó al fantasma? ¡Porque no tenía historial crediticio!",
    "¿Qué le dijo el riesgo de crédito al cliente? ¡No te preocupes, solo estoy evaluando tu *score*!",
    "¿Por qué el analista de crédito está soltero? Porque siempre ve demasiado riesgo en las relaciones."
]


## Definicion funciones

In [ ]:
def extract_text_with_pypdf2(path: str) -> str:
    text = ""
    with open(path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            page_text = page.extract_text()
            page_text=page_text.lower()
            if page_text:
                text += page_text + "\n"
    return text.strip()

# ─────── GUARDADO DE CONVERSACIÓN ───────
def save_conversation(user_input: str, response: str, log_path: Path = Path("chat_history.txt")):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(log_path, "a", encoding="utf-8") as f:
        f.write(f"[{timestamp}] Usuario: {user_input}\n")
        f.write(f"[{timestamp}] Chatbot: {response}\n\n")

# ─────── CONSTRUCCIÓN DE VECTORSTORE ───────
def build_vectorstore(text: str):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=200,
        separators=["\n\n", "\n", ".","-",":"]
    )
    chunks = splitter.split_text(text)
    embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)
    return FAISS.from_texts(chunks, embeddings)

# ─────── CARGA DEL MODELO LLM ───────
def build_llm_pipeline():
    tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL)
    model = AutoModelForSeq2SeqLM.from_pretrained(LLM_MODEL)
    pipe = pipeline(
        "text2text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=150
    )
    return HuggingFacePipeline(pipeline=pipe)

# ─────── CREACIÓN DE QA CHAIN ───────
def build_qa_chain(vector_store, llm):
    prompt = PromptTemplate(
        input_variables=["context", "question"],
        template="""
Responde como experto colombiano en analisis financiero. 
da respuestas cortas y concretas,se amistoso, y ten capacidad para conectar con los demas,
Usa únicamente la información del contexto.

Contexto:
{context}

Pregunta:
{question}

Respuesta:
"""
    )
    return RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vector_store.as_retriever(search_kwargs={"k": 3}),
        return_source_documents=True,
        chain_type_kwargs={"prompt": prompt, "document_variable_name": "context"}
    )

# ─────── RESPUESTA DEL CHATBOT ───────
def chatbot_response(user_input: str, qa_chain) -> str:
    input_lc = user_input.lower()
    if input_lc.startswith(("hola", "buenas", "qué tal", "cómo estás", "como estas")):
        response = "¡Hola! Estoy aquí para ayudarte con lo que necesites sobre riesgo de crédito en Colombia. ¿Qué deseas saber?"
    elif "chiste" in input_lc:
        response = random.choice(financial_jokes)
    else:
        try:
            result = qa_chain({"query": user_input})
            response = result.get("result", "").strip()
            if not response:
                response = "No encontré información suficiente en el documento. ¿Podrías reformular tu pregunta?"
        except Exception as e:
            response = f"💥 Hubo un error procesando tu pregunta: {str(e)}"
    save_conversation(user_input, response)
    return response

In [8]:


# ─────── EJECUCIÓN PRINCIPAL ───────
def main():
    print("📄 Extrayendo contenido del PDF...")
    context = extract_text_with_pypdf2(PDF_PATH)
    if not context:
        print("⚠️ No se pudo procesar el PDF.")
        return

    print("🔎 Construyendo base de conocimiento vectorial...")
    vector_store = build_vectorstore(context)

    print("🧠 Cargando modelo de lenguaje...")
    llm = build_llm_pipeline()

    print("🔗 Preparando sistema de preguntas y respuestas...")
    qa_chain = build_qa_chain(vector_store, llm)

    print("🤖 Asistente listo. Escribe 'salir' para terminar.")
    while True:
        user_input = input("Tú: ").strip()
        if user_input.lower() == "salir":
            print("👋 ¡Hasta pronto!")
            break
        print("Chatbot:", chatbot_response(user_input, qa_chain))

if __name__ == "__main__":
    main()

Device set to use cpu


 hola


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Eres un asistente financiero que responde solo con información precisa basada en los datos proporcionados. Usa el contexto para responder preguntas financieras de forma clara y directa. Si la pregunta no es financiera, redirige amigablemente al tema financiero.

Pregunta: hola
Información: Entidad: Bancamía, Producto: CDT, Plazo: 180 días, Tasa E.A.: 10.69%, Monto Mínimo: 200000 COP, Actualizado: 2025-01-05

Entidad: Banco Unión, Producto: CDT, Plazo: 360 días, Tasa E.A.: 10.42%, Monto Mínimo: 1000000 COP, Actualizado: 2024-11-01

Entidad: Banco Pichincha, Producto: Crédito Hipotecario, Plazo: 3600 días, Tasa E.A.: 8.5%, Monto Mínimo: 50000000 COP, Actualizado: 2025-01-05

Entidad: Ban100, Producto: CDT, Plazo: 360 días, Tasa E.A.: 11.38%, Monto Mínimo: 1000000 COP, Actualizado: 2024-11-01
Respuesta:  No, No hay información disponible, Redirige a otro tema


 buen dia


Eres un asistente financiero que responde solo con información precisa basada en los datos proporcionados. Usa el contexto para responder preguntas financieras de forma clara y directa. Si la pregunta no es financiera, redirige amigablemente al tema financiero.

Pregunta: Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: hola
Assistant: Eres un asistente financiero que responde solo con información precisa basada en los datos proporcionados. Usa el contexto para responder preguntas financieras de forma clara y directa. Si la pregunta no es financiera, redirige amigablemente al tema financiero.

Pregunta: hola
Información: Entidad: Bancamía, Producto: CDT, Plazo: 180 días, Tasa E.A.: 10.69%, Monto Mínimo: 200000 COP, Actualizado: 2025-01-05

Entidad: Banco Unión, Producto: CDT, Plazo: 360 días, Tasa E.A.: 10.42%, Monto Mínimo: 1000000 COP, Actualizado: 2024-11-01

Entid

 salir
